In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
import shapefile  
import datetime
import math
from sklearn.linear_model import LassoCV
from sklearn import linear_model
from random import sample
from sklearn.decomposition import PCA


# 0. settings

In [ ]:
dataSaveDIR = # where you save and load data
ety = pd.read_csv(dataSaveDIR+filename)
acs6 = pd.read_csv(dataSaveDIR+filename)

cate1 = pd.read_csv(dataSaveDIR+filename)
cate2 = pd.read_csv(dataSaveDIR+filename)
cate3 = pd.read_csv(dataSaveDIR+filename)
cate4 = pd.read_csv(dataSaveDIR+filename)
cate5 = pd.read_csv(dataSaveDIR+filename)
comp = pd.read_csv(dataSaveDIR+filename)

cate1r = pd.read_csv(dataSaveDIR+filename)
cate2r = pd.read_csv(dataSaveDIR+filename)
cate3r = pd.read_csv(dataSaveDIR+filename)
cate4r = pd.read_csv(dataSaveDIR+filename)
cate5r = pd.read_csv(dataSaveDIR+filename)
compr = pd.read_csv(dataSaveDIR+filename)

In [ ]:
# major parameters
mile_R = # earth radius in miles
meter_R = # earth radius in meters

sampStxt = np.loadtxt(dataSaveDIR+'sampS.txt')
sampS = [int(item) for item in sampStxt]

# de-close thresholds
decloseTH = 8 #1 de-close threshold, CF-real, unit: 0.025 miles
ratiothresh = 8 #20 de-close threshold, CF-CF, unit: 0.025 miles

actFiltNumb = 80

pcadim = 30 # PCA dimension

etylat = #entry latitude label name
etylon = #entry longitude label name
etyopendate = #entry opening date label name


In [ ]:

CNNOutcomeFileName = # all generated counterfactual location
fname = dataSaveDIR + CNNOutcomeFileName
pred1 = pd.read_csv(fname)


In [ ]:
# calculate and verify basic features of CNN outcome

nbpred = len(pred1.index) # number of counterfactual locations given by CNN
print(nbpred)
nbpred101 = int(nbpred/101) # number of center points used by CNN
print(nbpred101)

neg,pos = 0,0 # number of negative\positive locations given by CNN
poscnt = 0 # number of real missing points in positive locations
negcnt = 0 # number of non-real missing points in negative locations
for i in range(nbpred101):
    if pred1.loc[i*101,'S_id']<1000:
        pos+=1
        if pred1.loc[i*101,'real_missing']==True:
            poscnt+=1
    if pred1.loc[i*101,'S_id']>1000:
        neg+=1
        if pred1.loc[i*101,'real_missing']==False:
            negcnt+=1  

In [ ]:
# functions
def IDtoInfo(id):
    # Enter S_ID, return the required information
    if id<1000:
        lat = ety.loc[id,etylat] # gros-->ety
        lont = ety.loc[id,etylon]
    if id>1000:
        cnt = id-1001
        ind = sampS[cnt]
        lat = acs6.loc[ind,'LATITUDE'] # data --> acs6
        lont = acs6.loc[ind,'LONGITUDE']
        
    return lat, lont

def relaloc(lat,lont,x,y):
    '''Enter the reference point latitude, longtitude, and the XY position of the target point relative to the reference point, 
        and return the absolute lat, lont of the target point
    '''
    R = 6371393
    xrad = x/R
    yrad = y/R
    #print(xrad,yrad)
    xdeg = xrad*180/np.pi
    ydeg = yrad*180/np.pi 
    
    newlat = lat+ydeg
    newlont = lont+xdeg
    
    return newlat, newlont

# Haversine formula
def linedis(lat1,lon1,lat2,lon2):
    # Enter the latitude and longitude of the angle system and return the distance in meters
    R = meter_R

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return distance

# 1. process on "missing T/F"

In [ ]:
trd = []
frd = []
tcnt,fcnt = 0,0

for i in range(nbpred101):
    if i % 100==0:
        print(i) # For estimating finishing time
    if pred1.loc[i*101,'real_missing']== True:
        rx = pred1.loc[i*101:i*101+99,'x']
        ry = pred1.loc[i*101:i*101+99,'y']
        rr = rx*rx+ry*ry
        
        ra = pred1.loc[i*101:i*101+99,'activation']
        rae = np.exp(ra)
        
        ras = np.sum(ra)
        raes = np.sum(rae)

        ind = rr.nsmallest(1).index.values[0]
        
        rlat,rlont = IDtoInfo(pred1.loc[ind,'S_id'])
        relat,relont = relaloc(rlat, rlont, pred1.loc[ind,'x'], pred1.loc[ind,'y'])

        line = [tcnt,relat,relont,ra[ind],ra[ind]/ras,rae[ind],rae[ind]/raes]
        trd.append(line)
        
        tcnt+=1
    else:
        rx = pred1.loc[i*101:i*101+99,'x']
        ry = pred1.loc[i*101:i*101+99,'y']
        rr = rx*rx+ry*ry
        
        ra = pred1.loc[i*101:i*101+99,'activation']
        rae = np.exp(ra)
        
        ras = np.sum(ra)
        raes = np.sum(rae)
        
        for j in range(100):
            ind = i*101+j #ra.nlargest(1).index.values[0]

            rlat,rlont = IDtoInfo(pred1.loc[ind,'S_id'])
            relat,relont = relaloc(rlat, rlont, pred1.loc[ind,'x'], pred1.loc[ind,'y'])

            line = [fcnt, relat, relont, ra[ind], ra[ind]/ras, rae[ind], rae[ind]/raes]
            frd.append(line)

            fcnt+=1

In [ ]:
# Step 1: view the filtered real store information
trd1 = pd.DataFrame(data=trd, index=range(tcnt), 
                    columns=['No','latitude','longitude','activation','acvt percent','exp acvt','exp acvt percent'])

# Step 2: view the filtered fake store information
frd1 = pd.DataFrame(data=frd, index=range(fcnt), 
                    columns=['No','latitude','longitude','activation','acvt percent','exp acvt','exp acvt percent'])



# 2. Delete CF-real points that are too close


In [ ]:
# Delete CF-real points that are too close

delta_radious = 0.025/mile_R  # Bin radious
delta_degrees = math.degrees(delta_radious)
bingap=0.025*1609.34

trd3 = ety.copy(deep=True)
trd3 = trd3[['No',etylat,etylon]]
trd3.columns = ['No','latitude','longitude']
droplist = []
for i in trd3['No'].values:
    print(i)
    fgslat = trd3.loc[i,'latitude']  # fgs = false grocesry store
    fgslot = trd3.loc[i,'longitude']
    
   
    # Delete CF-real points that are too close
    fgsCtbin1 = frd1[frd1['latitude']>=fgslat-decloseTH*delta_degrees].copy(deep=True)
    fgsCtbin2 = fgsCtbin1[fgsCtbin1['latitude']<=fgslat+decloseTH*delta_degrees].copy(deep=True)
    
    fgsCtbin3 = fgsCtbin2[fgsCtbin2['longitude']>=fgslot-decloseTH*delta_degrees].copy(deep=True)
    fgsCtbin4 = fgsCtbin3[fgsCtbin3['longitude']<=fgslot+decloseTH*delta_degrees].copy(deep=True)
        
    if len(fgsCtbin4.index)>=1:
        print('real:',len(fgsCtbin4.index),'index:',i)
        droplist.append(list(fgsCtbin4.index))
    

In [ ]:
func = lambda x: [y for l in x for y in func(l)] if type(x) is list else [x]
droplist1 = func(droplist)
droplist2 = list(set(droplist1))
frd1 = frd1.drop(droplist2)


# 3. First round of screening based on activation scores


In [ ]:
# First round of screening based on activation scores
keepCloseTH = 500
trd3 = ety.copy(deep=True)
trd3 = trd3[['No',etylat,etylon]]
trd3.columns = ['No','latitude','longitude']

idlist = []
keeplist = []
idcnt = []
idless20 = []
flag = 0
for i in trd3['No'].values:
    #print(i)
    #flag = True # whether i is still in tfrd33
    fgslat = trd3.loc[i,'latitude']  # fgs = false grocesry store
    fgslot = trd3.loc[i,'longitude']
   
    #filter out CF-reals that are of proper distance
    fgsCtbin1 = frd1[frd1['latitude']>=fgslat-keepCloseTH*delta_degrees].copy(deep=True)
    fgsCtbin2 = fgsCtbin1[fgsCtbin1['latitude']<=fgslat+keepCloseTH*delta_degrees].copy(deep=True)
    
    fgsCtbin3 = fgsCtbin2[fgsCtbin2['longitude']>=fgslot-keepCloseTH*delta_degrees].copy(deep=True)
    fgsCtbin4 = fgsCtbin3[fgsCtbin3['longitude']<=fgslot+keepCloseTH*delta_degrees].copy(deep=True)
    if len(fgsCtbin4.index)==0:
        print('real number _{} has no match'.format(i))
        
    elif len(fgsCtbin4.index)<=actFiltNumb:
        keeplist.append(list(fgsCtbin4.index))
        
newf2 = []
for i in range(len(keeplist)):
    temp = keeplist[i]
    for j in range(len(temp)):
        newf2.append(temp[j])   
        
for i in range(tcnt):
    print(i)
    acvt = trd1.loc[i,'activation']
    expper = trd1.loc[i,'exp acvt percent']
    
    facvt = frd1['activation'].copy(deep=True)
    fexpper = frd1['exp acvt percent'].copy(deep=True)
    
    facvt = np.abs(facvt - acvt)
    fexpper = np.abs(fexpper - expper)
    
    ind1 = facvt.nsmallest(20).index.values
    ind2 = fexpper.nsmallest(20).index.values
    
    if i==0:
        newf = np.concatenate((ind1,ind2),axis = 0)
    else:
        newf = np.concatenate((newf,ind1,ind2),axis = 0)
        
newf = list(set(newf))
newf = newf + newf2
newf = list(set(newf))

frd2 = frd1.copy(deep=True)
frd2 = frd2.loc[newf]
frd2 = frd2.sort_values(by = "No")


# 4. 1st round propensity score matching

In [ ]:
acs7 = acs6.copy()
acs7.insert(18, 'minorRace', acs7.pop('minorRace'))

frd3 = frd2.copy(deep=True)
# leave blank for acs feature
for i in range(10*(len(acs6.columns)-9)):
    frd3['binA{}'.format(i+1)] = np.zeros(len(frd2.index))
    
# leave blank for cate feature
for i in range(10*(len(acs6.columns)-9), 10*(len(acs6.columns)-9)+10*5, 1):
    frd3['binB{}'.format(i+1)] = np.zeros(len(frd2.index))
    
# leave blank for comp feature
for i in range(10*(len(acs6.columns)-9)+10*5, 10*(len(acs6.columns)-9)+10*5+10*1, 1):
    frd3['binC{}'.format(i+1)] = np.zeros(len(frd2.index))


In [ ]:
# ct100 -> acs7

delta_radious = 0.025/mile_R  # Bin radious 0.025
delta_degrees = math.degrees(delta_radious)
bingap=delta_radious*mile_R*1609.34

bin_radious = 0.025/mile_R  # Bin radious 0.025
bin_degrees = math.degrees(bin_radious)
bingap2=bin_radious*mile_R*1609.34

for i in frd3['No'].values:
    # acs7
    fgslat = frd3.loc[i,'latitude']  # fgs = false grocesry store
    fgslot = frd3.loc[i,'longitude']

    fgsCtbin = acs7[acs7['LATITUDE']>=fgslat-10*bin_degrees].copy(deep=True)
    fgsCtbin = fgsCtbin[fgsCtbin['LATITUDE']<=fgslat+10*bin_degrees]

    fgsCtbin = fgsCtbin[fgsCtbin['LONGITUDE']>=fgslot-10*bin_degrees]
    fgsCtbin = fgsCtbin[fgsCtbin['LONGITUDE']<=fgslot+10*bin_degrees]

    nb_fgsCtbin = len(fgsCtbin.index)

    for j in fgsCtbin.index.values:
        tempdis = linedis(fgslat,fgslot,fgsCtbin.loc[j,'LATITUDE'],fgsCtbin.loc[j,'LONGITUDE'])
        inx = math.ceil(tempdis/bingap2)
        if inx<=10:
            frd3.loc[i,'binA{}'.format(1+(inx-1)*18):'binA{}'.format((inx)*18)]+=np.array(acs7.iloc[j,1:19].copy())

    # cate1
    denos = np.zeros(10)
    catebin = cate1r[cate1r['poi_lat']>=fgslat-10*bin_degrees].copy(deep=True)
    catebin = catebin[catebin['poi_lat']<=fgslat+10*bin_degrees]

    catebin = catebin[catebin['poi_lon']>=fgslot-10*bin_degrees]
    catebin = catebin[catebin['poi_lon']<=fgslot+10*bin_degrees]

    for j in catebin.index.values:
        tempdis = linedis(fgslat,fgslot,catebin.loc[j,'poi_lat'],catebin.loc[j,'poi_lon'])
        inx2 = math.ceil(tempdis/bingap2)
        #print('inx2',inx2)
        if inx2<=10:
            #
            denos[inx2-1]+=1

    frd3.loc[i,'binB{}'.format(181):'binB{}'.format(190)]+= denos

    # cate2
    denos = np.zeros(10)
    catebin = cate2r[cate2r['poi_lat']>=fgslat-10*bin_degrees].copy(deep=True)
    catebin = catebin[catebin['poi_lat']<=fgslat+10*bin_degrees]

    catebin = catebin[catebin['poi_lon']>=fgslot-10*bin_degrees]
    catebin = catebin[catebin['poi_lon']<=fgslot+10*bin_degrees]

    for j in catebin.index.values:
        tempdis = linedis(fgslat,fgslot,catebin.loc[j,'poi_lat'],catebin.loc[j,'poi_lon'])
        inx2 = math.ceil(tempdis/bingap2)
        #print('inx2',inx2)
        if inx2<=10:
            #
            denos[inx2-1]+=1

    frd3.loc[i,'binB{}'.format(191):'binB{}'.format(200)]+= denos

    # cate3
    denos = np.zeros(10)
    catebin = cate3r[cate3r['poi_lat']>=fgslat-10*bin_degrees].copy(deep=True)
    catebin = catebin[catebin['poi_lat']<=fgslat+10*bin_degrees]

    catebin = catebin[catebin['poi_lon']>=fgslot-10*bin_degrees]
    catebin = catebin[catebin['poi_lon']<=fgslot+10*bin_degrees]

    for j in catebin.index.values:
        tempdis = linedis(fgslat,fgslot,catebin.loc[j,'poi_lat'],catebin.loc[j,'poi_lon'])
        inx2 = math.ceil(tempdis/bingap2)
        #print('inx2',inx2)
        if inx2<=10:
            #
            denos[inx2-1]+=1

    frd3.loc[i,'binB{}'.format(201):'binB{}'.format(210)]+= denos

    # cate4
    denos = np.zeros(10)
    catebin = cate4r[cate4r['poi_lat']>=fgslat-10*bin_degrees].copy(deep=True)
    catebin = catebin[catebin['poi_lat']<=fgslat+10*bin_degrees]

    catebin = catebin[catebin['poi_lon']>=fgslot-10*bin_degrees]
    catebin = catebin[catebin['poi_lon']<=fgslot+10*bin_degrees]

    for j in catebin.index.values:
        tempdis = linedis(fgslat,fgslot,catebin.loc[j,'poi_lat'],catebin.loc[j,'poi_lon'])
        inx2 = math.ceil(tempdis/bingap2)
        #print('inx2',inx2)
        if inx2<=10:
            #
            denos[inx2-1]+=1

    frd3.loc[i,'binB{}'.format(211):'binB{}'.format(220)]+= denos

    # cate5
    denos = np.zeros(10)
    catebin = cate5r[cate5r['poi_lat']>=fgslat-10*bin_degrees].copy(deep=True)
    catebin = catebin[catebin['poi_lat']<=fgslat+10*bin_degrees]

    catebin = catebin[catebin['poi_lon']>=fgslot-10*bin_degrees]
    catebin = catebin[catebin['poi_lon']<=fgslot+10*bin_degrees]
    #print('Ct in the bin:',len(catebin.index))#,ctgsbin)

    for j in catebin.index.values:
        tempdis = linedis(fgslat,fgslot,catebin.loc[j,'poi_lat'],catebin.loc[j,'poi_lon'])
        inx2 = math.ceil(tempdis/bingap2)
        #print('inx2',inx2)
        if inx2<=10:
            #
            denos[inx2-1]+=1

    frd3.loc[i,'binB{}'.format(221):'binB{}'.format(230)]+= denos

    # comp
    denos = np.zeros(10)
    compbin = compr[compr['poi_lat']>=fgslat-10*bin_degrees].copy(deep=True)
    compbin = compbin[compbin['poi_lat']<=fgslat+10*bin_degrees]

    compbin = compbin[compbin['poi_lon']>=fgslot-10*bin_degrees]
    compbin = compbin[compbin['poi_lon']<=fgslot+10*bin_degrees]

    for j in compbin.index.values:
        tempdis = linedis(fgslat,fgslot,compbin.loc[j,'poi_lat'],compbin.loc[j,'poi_lon'])
        inx3 = math.ceil(tempdis/bingap2)
        if inx3<=10:
            denos[inx3-1]+=1

    frd3.loc[i,'binC{}'.format(231):]+= denos
    
frd3['realmiss'] = [0]*len(frd3.index)

In [ ]:

trd3 = ety.copy(deep=True)
#trd3 = ety.iloc[0:215,:].copy(deep=True)
trd3 = trd3[['No',etylat,etylon]]
trd3.columns = ['No','latitude','longitude']

tcnt_trd3 = len(trd3.index)
trd3['activation'] = np.zeros(tcnt_trd3)
trd3['acvt percent'] = np.zeros(tcnt_trd3)
trd3['exp acvt'] = np.zeros(tcnt_trd3)
trd3['exp acvt percent'] = np.zeros(tcnt_trd3)

# leave blank for acs feature
for i in range(10*(len(acs6.columns)-9)):
    trd3['binA{}'.format(i+1)] = np.zeros(tcnt_trd3)
    
# leave blank for cate feature
for i in range(10*(len(acs6.columns)-9), 10*(len(acs6.columns)-9)+10*5, 1):
    trd3['binB{}'.format(i+1)] = np.zeros(tcnt_trd3)
    
# leave blank for comp feature
for i in range(10*(len(acs6.columns)-9)+10*5, 10*(len(acs6.columns)-9)+10*5+10*1, 1):
    trd3['binC{}'.format(i+1)] = np.zeros(tcnt_trd3)


In [ ]:

for i in trd3['No'].values:
    print(i)
    # acs7
    fgslat = trd3.loc[i,'latitude']  # fgs = false grocesry store
    fgslot = trd3.loc[i,'longitude']

    fgsCtbin = acs7[acs7['LATITUDE']>=fgslat-10*bin_degrees].copy(deep=True)
    fgsCtbin = fgsCtbin[fgsCtbin['LATITUDE']<=fgslat+10*bin_degrees]

    fgsCtbin = fgsCtbin[fgsCtbin['LONGITUDE']>=fgslot-10*bin_degrees]
    fgsCtbin = fgsCtbin[fgsCtbin['LONGITUDE']<=fgslot+10*bin_degrees]

    #print('Ct in the bin:',len(fgsCtbin.index))#,ctgsbin)
    nb_fgsCtbin = len(fgsCtbin.index)

    for j in fgsCtbin.index.values:
        tempdis = linedis(fgslat,fgslot,fgsCtbin.loc[j,'LATITUDE'],fgsCtbin.loc[j,'LONGITUDE'])
        inx = math.ceil(tempdis/bingap2)
        #print('inx',inx)
        if inx<=10:
            trd3.loc[i,'binA{}'.format(1+(inx-1)*18):'binA{}'.format((inx)*18)]+=list(acs7.iloc[j,1:19].copy())

    # cate1
    denos = np.zeros(10)
    catebin = cate1[cate1['poi_lat']>=fgslat-10*bin_degrees].copy(deep=True)
    catebin = catebin[catebin['poi_lat']<=fgslat+10*bin_degrees]

    catebin = catebin[catebin['poi_lon']>=fgslot-10*bin_degrees]
    catebin = catebin[catebin['poi_lon']<=fgslot+10*bin_degrees]

    for j in catebin.index.values:
        tempdis = linedis(fgslat,fgslot,catebin.loc[j,'poi_lat'],catebin.loc[j,'poi_lon'])
        inx2 = math.ceil(tempdis/bingap2)
        #print('inx2',inx2)
        if inx2<=10:
            #
            denos[inx2-1]+=1

    trd3.loc[i,'binB{}'.format(181):'binB{}'.format(190)]+= denos

    # cate2
    denos = np.zeros(10)
    catebin = cate2[cate2['poi_lat']>=fgslat-10*bin_degrees].copy(deep=True)
    catebin = catebin[catebin['poi_lat']<=fgslat+10*bin_degrees]

    catebin = catebin[catebin['poi_lon']>=fgslot-10*bin_degrees]
    catebin = catebin[catebin['poi_lon']<=fgslot+10*bin_degrees]

    for j in catebin.index.values:
        tempdis = linedis(fgslat,fgslot,catebin.loc[j,'poi_lat'],catebin.loc[j,'poi_lon'])
        inx2 = math.ceil(tempdis/bingap2)
        #print('inx2',inx2)
        if inx2<=10:
            #
            denos[inx2-1]+=1

    trd3.loc[i,'binB{}'.format(191):'binB{}'.format(200)]+= denos

    # cate3
    denos = np.zeros(10)
    catebin = cate3[cate3['poi_lat']>=fgslat-10*bin_degrees].copy(deep=True)
    catebin = catebin[catebin['poi_lat']<=fgslat+10*bin_degrees]

    catebin = catebin[catebin['poi_lon']>=fgslot-10*bin_degrees]
    catebin = catebin[catebin['poi_lon']<=fgslot+10*bin_degrees]

    for j in catebin.index.values:
        tempdis = linedis(fgslat,fgslot,catebin.loc[j,'poi_lat'],catebin.loc[j,'poi_lon'])
        inx2 = math.ceil(tempdis/bingap2)
        #print('inx2',inx2)
        if inx2<=10:
            #
            denos[inx2-1]+=1

    trd3.loc[i,'binB{}'.format(201):'binB{}'.format(210)]+= denos

    # cate4
    denos = np.zeros(10)
    catebin = cate4[cate4['poi_lat']>=fgslat-10*bin_degrees].copy(deep=True)
    catebin = catebin[catebin['poi_lat']<=fgslat+10*bin_degrees]

    catebin = catebin[catebin['poi_lon']>=fgslot-10*bin_degrees]
    catebin = catebin[catebin['poi_lon']<=fgslot+10*bin_degrees]

    for j in catebin.index.values:
        tempdis = linedis(fgslat,fgslot,catebin.loc[j,'poi_lat'],catebin.loc[j,'poi_lon'])
        inx2 = math.ceil(tempdis/bingap2)
        #print('inx2',inx2)
        if inx2<=10:
            #
            denos[inx2-1]+=1

    trd3.loc[i,'binB{}'.format(211):'binB{}'.format(220)]+= denos

    # cate5
    denos = np.zeros(10)
    catebin = cate5[cate5['poi_lat']>=fgslat-10*bin_degrees].copy(deep=True)
    catebin = catebin[catebin['poi_lat']<=fgslat+10*bin_degrees]

    catebin = catebin[catebin['poi_lon']>=fgslot-10*bin_degrees]
    catebin = catebin[catebin['poi_lon']<=fgslot+10*bin_degrees]
    #print('Ct in the bin:',len(catebin.index))#,ctgsbin)

    for j in catebin.index.values:
        tempdis = linedis(fgslat,fgslot,catebin.loc[j,'poi_lat'],catebin.loc[j,'poi_lon'])
        inx2 = math.ceil(tempdis/bingap2)
        #print('inx2',inx2)
        if inx2<=10:
            #
            denos[inx2-1]+=1

    trd3.loc[i,'binB{}'.format(221):'binB{}'.format(230)]+= denos

    # comp
    denos = np.zeros(10)
    compbin = comp[comp['poi_lat']>=fgslat-10*bin_degrees].copy(deep=True)
    compbin = compbin[compbin['poi_lat']<=fgslat+10*bin_degrees]

    compbin = compbin[compbin['poi_lon']>=fgslot-10*bin_degrees]
    compbin = compbin[compbin['poi_lon']<=fgslot+10*bin_degrees]

    for j in compbin.index.values:
        tempdis = linedis(fgslat,fgslot,compbin.loc[j,'poi_lat'],compbin.loc[j,'poi_lon'])
        inx3 = math.ceil(tempdis/bingap2)
        #print('inx3',inx3)
        if inx3<=10:
            #
            denos[inx3-1]+=1

    trd3.loc[i,'binC{}'.format(231):]+= denos
    
trd3['realmiss'] = [1]*len(trd3.index)

# 5. PCA and 1st round propensity score matching

In [ ]:
# Delete Null frd3 and trd3 
droplist = []
nolist = frd3.index.values
tnolist = trd3.index.values
nocnt = 0
nocnt2 = 0

for i in range(len(nolist)):
    if (np.sum(frd3.iloc[i,-241:]))==0:
        droplist.append(frd3.loc[nolist[i],'No'])
        nocnt+=1

for i in range(len(tnolist)):
    if (np.sum(trd3.iloc[i,-241:]))==1:
        droplist.append(trd3.loc[tnolist[i],'No'])
        nocnt2+=1
       
print('all zero:',nocnt,'acs zero(first A180):',nocnt2)
print(droplist)


In [ ]:
# drop according to droplist
frd3 = frd3.drop(droplist[0:nocnt])
trd3 = trd3.drop(droplist[nocnt:])

# combine to prepare propensity input
tfrd1 = trd3.append(frd3).copy(deep=True)


In [ ]:
# try to use PCA before propensity score

pca = PCA(n_components=pcadim)
trainset = tfrd1.iloc[0:len(trd3.index)-nocnt2,7:-1].copy()
pca.fit(trainset)

testset = tfrd1.iloc[:,7:-1].copy()
pcaout = pca.transform(testset)
pcaout = np.array(pcaout)


In [ ]:
tfrd2 = tfrd1[['No','latitude','longitude','realmiss']].copy()
for i in range(pcadim):
    tfrd2['bin{}'.format(i)] = np.zeros(len(tfrd1.index))
tfrd2.loc[:,'bin0':'bin{}'.format(pcadim-1)] = pcaout


In [ ]:
# propensity score matching

# unbale to optimize thouroughly
formula = "realmiss ~ bin0 + bin1 + bin2 + bin3 + bin4 + bin5 + bin6 + bin7 + bin8 + bin9 + bin10 + bin11 + bin12 + bin13 + bin14 + bin15 + bin16 + bin17 + bin18 + bin19 + bin20 + bin21 + bin22 + bin23 + bin24 + bin25 + bin26 + bin27 + bin28 + bin29"
model = sm.Logit.from_formula(formula, data = tfrd2)
re = model.fit(maxiter=100)
X = tfrd2[['bin0', 'bin1', 'bin2', 'bin3', 'bin4', 'bin5', 'bin6', 'bin7', 'bin8', 'bin9', 'bin10', 'bin11', 'bin12', 'bin13', 'bin14', 'bin15', 'bin16', 'bin17', 'bin18', 'bin19', 'bin20', 'bin21', 'bin22', 'bin23', 'bin24', 'bin25', 'bin26', 'bin27', 'bin28', 'bin29']]
tfrd2['ps'] = re.predict(X)


In [ ]:
tfrd3 = tfrd2.copy(deep=True)

tfrd31 = tfrd3.iloc[:len(trd3.index),:].copy(deep=True) # real store
tfrd32 = tfrd3.iloc[len(trd3.index):,:].copy(deep=True) # counterfactual store

tfrd32.index = tfrd32['No'].values


In [ ]:
# Draw preliminary comparison diagram, before matching
p1=sns.kdeplot(tfrd31['ps'], shade=True, color="b",label='real openings') # real
p1=sns.kdeplot(tfrd32['ps'], shade=True, color="r",label='counterfactual') # counterfactual
plt.legend()

In [ ]:
# Screening with P-score, the first round of matching
zerocnt = 0
rangedegrees = 400 # 10 miles
fpscolect = []
finalf = np.zeros(0)
for i in range(len(tfrd31.index)):
    if i not in droplist:
        tps = tfrd31.loc[i,'ps']
        tlat = tfrd31.loc[i,'latitude']
        tlon = tfrd31.loc[i,'longitude']

        fps0 = tfrd32.copy(deep=True)

        fps1 = fps0[fps0['latitude']>=tlat-rangedegrees*delta_degrees].copy(deep=True) # 1000 delta_degrees = 25 miles
        fps1 = fps1[fps1['latitude']<=tlat+rangedegrees*delta_degrees]

        fps1 = fps1[fps1['longitude']>=tlon-rangedegrees*delta_degrees]
        fps1 = fps1[fps1['longitude']<=tlon+rangedegrees*delta_degrees]

        fps = fps1['ps'].copy(deep=True)
        fpscolect.append(len(fps))
        if len(fps)==0:
            zerocnt+=1
        difsign = np.sign(fps-tps)
        difabs = np.abs(fps-tps)

        ind = difabs.nsmallest(10).index.values

        if len(finalf)==0:
            finalf = np.array(ind)
        else:
            finalf = np.concatenate((finalf,ind),axis = 0)

finalf = list(set(finalf))

sns.histplot(fpscolect, kde = True, color = 'blue', bins = 50, label='real-CF matching histogram') 
plt.xlabel('number of CFs matched')
plt.legend()

In [ ]:
# After 1st round of matching, delete pairs that are too close

tfrd33 = tfrd32.loc[finalf].copy(deep=True)

# Draw the diagram, before deleting points that are too close together
p1=sns.kdeplot(tfrd31['ps'], shade=True, color="b",label='real openings') # real
p1=sns.kdeplot(tfrd33['ps'], shade=True, color="r",label='counterfactual') # counterfactual
plt.legend()

In [ ]:
# After 1st round of matching, delete pairs that are too close

tfrd33_drop = tfrd33.copy(deep=True)

for i in tfrd33['No'].values:
    flag = True # whether i is still in tfrd33
    fgslat = tfrd33.loc[i,'latitude']  # fgs = false grocesry store
    fgslot = tfrd33.loc[i,'longitude']
    
    # Delete CF points that are too close to each other
    fgsCtbin1 = tfrd33_drop[tfrd33_drop['latitude']>=fgslat-ratiothresh*delta_degrees].copy(deep=True)
    fgsCtbin2 = fgsCtbin1[fgsCtbin1['latitude']<=fgslat+ratiothresh*delta_degrees].copy(deep=True)
    
    fgsCtbin3 = fgsCtbin2[fgsCtbin2['longitude']>=fgslot-ratiothresh*delta_degrees].copy(deep=True)
    fgsCtbin4 = fgsCtbin3[fgsCtbin3['longitude']<=fgslot+ratiothresh*delta_degrees].copy(deep=True)
        
    if len(fgsCtbin4.index)>1:
        print('CF:',len(fgsCtbin4.index),'index:',i)
        tfrd33_drop = tfrd33_drop.drop([i])
        flag = False

fgsCtbin = tfrd33_drop.copy(deep=True)

# Draw the diagram, after deleting the over close point
p1=sns.kdeplot(tfrd31['ps'], shade=True, color="b",label='real openings') # real
p1=sns.kdeplot(fgsCtbin['ps'], shade=True, color="r",label='counterfactual') # counterfactual
plt.legend()

# 6. 2nd round of propensity matching

In [ ]:
# combing information of real & counterfactual locations
tfrd100 = tfrd31.append(fgsCtbin).copy(deep=True)
tfrd100

# re-calculate p-score

formula = "realmiss ~bin0 + bin1 + bin2 + bin3 + bin4 + bin5 + bin6 + bin7 + bin8 + bin9 + bin10 + bin11 + bin12 + bin13 + bin14 + bin15 + bin16 + bin17 + bin18 + bin19 + bin20 + bin21 + bin22 + bin23 + bin24 + bin25 + bin26 + bin27 + bin28 + bin29"
model = sm.Logit.from_formula(formula, data = tfrd100)
re = model.fit()
X = tfrd100[['bin0', 'bin1', 'bin2', 'bin3', 'bin4', 'bin5', 'bin6', 'bin7', 'bin8', 'bin9', 'bin10', 'bin11', 'bin12', 'bin13', 'bin14', 'bin15', 'bin16', 'bin17', 'bin18', 'bin19', 'bin20', 'bin21', 'bin22', 'bin23', 'bin24', 'bin25', 'bin26', 'bin27', 'bin28', 'bin29']]
tfrd100['ps'] = re.predict(X)

tfrd31 = tfrd100.iloc[:len(trd3.index),:].copy(deep=True) # real store
fgsCtbin2 = tfrd100.iloc[len(trd3.index):,:].copy(deep=True) # counterfactual store

fgsCtbin2.index = fgsCtbin2['No'].values

In [ ]:
# Make the second round of match based on the new P-score
zerocnt2 = 0
rangedegrees2 = 600 
reallines, cflines = [],[]
finalf = np.zeros(0)

for i in range(len(tfrd31.index)):
    if i not in droplist:
        tps = tfrd31.loc[i,'ps']
        tlat = tfrd31.loc[i,'latitude']
        tlon = tfrd31.loc[i,'longitude']

        realline = [ety.loc[i,'sg_placekey'],tlat, tlon, 'R', tps]
        reallines.append(realline)

        fps0 = fgsCtbin2.copy(deep=True)

        fps1 = fps0[fps0['latitude']>=tlat-rangedegrees2*delta_degrees].copy(deep=True) # 1000 delta_degrees = 25 miles
        fps1 = fps1[fps1['latitude']<=tlat+rangedegrees2*delta_degrees]

        fps1 = fps1[fps1['longitude']>=tlon-rangedegrees2*delta_degrees]
        fps1 = fps1[fps1['longitude']<=tlon+rangedegrees2*delta_degrees]

        fps = fps1['ps'].copy(deep=True)
        print('i',i,'cand Numb:',len(fps))
        if len(fps)==0:
            zerocnt2+=1

        difsign = np.sign(fps-tps)
        difabs = np.abs(fps-tps)

        ind = difabs.nsmallest(1).index.values

        if len(finalf)==0:
            finalf = np.array(ind)
        
        else:
            for item in ind:
                cfline = [ety.loc[i,'sg_placekey'], fps1.loc[item, 'latitude'],fps1.loc[item, 'longitude'],'CF', fps1.loc[item, 'ps']]
                cflines.append(cfline)
                print(fps1.loc[item, 'ps'])

            finalf = np.concatenate((finalf,ind),axis = 0)

finalf = list(set(finalf))

tfrd34 = fgsCtbin2.loc[finalf].copy(deep=True)


In [ ]:
# Merge and store reallines and cflines

version = # version name

out = reallines+cflines
out = np.array(out)
out = pd.DataFrame(data=out, index=range(out.shape[0]), 
                    columns=['sg_placekey','latitude','longitude','Real_CF','ps'])

out.to_csv(dataSaveDIR+'ety_CF_{}.csv'.format(version)
           ,index=False,header=out.columns,sep=',')

In [ ]:
# Plot comparison diagram, After Matching
p1=sns.kdeplot(tfrd31['ps'], shade=True, color="b",label='real openings') # real
p1=sns.kdeplot(tfrd34['ps'], shade=True, color="r",label='counterfactual') # counterfactual
print('real:',len(tfrd31['ps']),',CF:',len(tfrd34['ps']),',zero cnd:',zerocnt2)
plt.xlabel('propensity score',fontsize=18)
plt.xticks(fontsize=15)

plt.ylabel('density',fontsize=18)
plt.yticks(fontsize=15)

plt.legend(fontsize=18)

In [ ]:
out = tfrd31.copy()
out.to_csv(dataSaveDIR+'tfrd31_v{}.csv'.format(version)
           ,index=False,header=out.columns,sep=',')

out = tfrd34.copy()
out.to_csv(dataSaveDIR+'tfrd34_v{}.csv'.format(version)
           ,index=False,header=out.columns,sep=',')